In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import os
import glob
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import  applications
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,Conv1D, MaxPool2D,GlobalAveragePooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

import warnings
warnings.filterwarnings('ignore')
import random
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)
import glob
import shutil
import os
import itertools
from sklearn.metrics import confusion_matrix ,classification_report

In [2]:
# import sys 
# !{sys.executable} -m pip install tensorflow==1.15.0

In [3]:
# import sys 
# !{sys.executable} -m pip install tensorflow==1.15.0

In [4]:
import tensorflow as tf
print(tf.__version__)

2.5.0


In [7]:
train_path='C:/Users/KIIT/OneDrive/Desktop/deploy/train'

In [8]:
train_batches=ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(directory=train_path,target_size= (224,224),batch_size=10,shuffle=False)

Found 359 images belonging to 3 classes.


In [10]:
test_path='C:/Users/KIIT/OneDrive/Desktop/deploy/test'

In [11]:
test_batches=ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(directory=test_path,target_size= (224,224),batch_size=10,shuffle=False)

Found 15 images belonging to 3 classes.


In [12]:
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

base = MobileNetV2(weights = 'imagenet', include_top = False, input_shape = (224, 224, 3))
tf.keras.backend.clear_session()
    
for layer in base.layers:
    layer.trainable =  False

mobilenet_model = Sequential()
mobilenet_model.add(base)
mobilenet_model.add(GlobalAveragePooling2D())

mobilenet_model.add(Dense(256, activation='relu'))
mobilenet_model.add(Dropout(0.1))

mobilenet_model.add(Dense(256, activation='relu'))
mobilenet_model.add(Dropout(0.1))
mobilenet_model.add(Dense(3, activation='softmax'))

In [13]:
mobilenet_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               327936    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 7

In [14]:
mobilenet_model.compile(optimizer=Adam(learning_rate=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])
results=mobilenet_model.fit(train_batches,epochs=10)  

Epoch 1/10
36/36 [==============================] - 9s 176ms/step - loss: 0.9012 - accuracy: 0.5989
Epoch 2/10
36/36 [==============================] - 8s 223ms/step - loss: 0.4478 - accuracy: 0.9025
Epoch 3/10
36/36 [==============================] - 8s 223ms/step - loss: 0.2394 - accuracy: 0.9471
Epoch 4/10
36/36 [==============================] - 8s 228ms/step - loss: 0.1598 - accuracy: 0.9582
Epoch 5/10
36/36 [==============================] - 8s 217ms/step - loss: 0.0997 - accuracy: 0.9861
Epoch 6/10
36/36 [==============================] - 8s 226ms/step - loss: 0.0673 - accuracy: 0.9944
Epoch 7/10
36/36 [==============================] - 8s 217ms/step - loss: 0.0497 - accuracy: 0.99720s - loss: 0.0459 - accuracy: 
Epoch 8/10
36/36 [==============================] - 8s 212ms/step - loss: 0.0370 - accuracy: 0.9972
Epoch 9/10
36/36 [==============================] - 8s 225ms/step - loss: 0.0312 - accuracy: 1.0000
Epoch 10/10
36/36 [==============================] - 8s 228ms/step - l

In [15]:
mobilenet_model.evaluate(test_batches)

2/2 [==============================] - 1s 125ms/step - loss: 0.0133 - accuracy: 1.0000


[0.01334297563880682, 1.0]

In [16]:
mobilenet_model.save('my_model.h5')